In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycaret

In [16]:
!pip install numpy==1.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import pycaret
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_absolute_error, roc_auc_score, f1_score, confusion_matrix

In [5]:
dpath = 'drive/MyDrive/Seculayer/OSC/data/'

train = pd.read_csv(dpath + 'train.csv')
test = pd.read_csv(dpath + 'test.csv')

In [7]:
def check_not_missing(train):
    total = len(train)
    not_list = []
    for item in train.columns:
        if (total*0.2)>train[item].isnull().sum():
            not_list.append(item)
    return not_list

not_list = check_not_missing(train)

In [ ]:
real = train[not_list]
real.info()

In [10]:
teacher_X = real.drop(['ID'],axis=1)

# teacher_y = real['Y_LABEL']

In [ ]:
teacher_X.info()

In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()

teacher_X['COMPONENT_ARBITRARY_cat'] = le1.fit_transform(teacher_X['COMPONENT_ARBITRARY'])
teacher_X['YEAR_cat'] = le2.fit_transform(teacher_X['YEAR'])
teacher_X.drop(['YEAR','COMPONENT_ARBITRARY'],axis=1,inplace=True)

teacher_X.info()

In [12]:
teacher_X.dropna(axis=1,inplace=True)

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_cat','YEAR_cat']

numeric_features = []
for item in teacher_X.columns:
    if item not in categorical_features:
        if item != 'Y_LABEL':
            numeric_features.append(item)

numeric_features

In [14]:
np.__version__

'1.21.6'

In [ ]:
from pycaret.classification import *

clf = setup(teacher_X, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.8,
            session_id = 42,
            use_gpu=True)

In [18]:
best5 = compare_models(fold = 5, sort = 'f1', n_select = 5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9554,0.8897,0.5287,0.9219,0.6715,0.6494,0.6787,0.486
gbc,Gradient Boosting Classifier,0.9551,0.8858,0.5256,0.9215,0.6687,0.6466,0.6764,3.112
rf,Random Forest Classifier,0.9549,0.8755,0.5040,0.9525,0.6581,0.6365,0.6742,1.782
ada,Ada Boost Classifier,0.9536,0.8699,0.5071,0.9211,0.6534,0.6308,0.6636,0.714
lr,Logistic Regression,0.9530,0.8690,0.4856,0.9434,0.6403,0.6180,0.6576,0.380
svm,SVM - Linear Kernel,0.9510,0.0000,0.4856,0.9071,0.6308,0.6072,0.6422,0.078
et,Extra Trees Classifier,0.9486,0.8731,0.4178,0.9704,0.5831,0.5603,0.6178,1.426
dt,Decision Tree Classifier,0.9163,0.7510,0.5513,0.5145,0.5321,0.4862,0.4867,0.204
nb,Naive Bayes,0.8680,0.8169,0.6303,0.3586,0.4534,0.3857,0.4077,0.024
knn,K Neighbors Classifier,0.9313,0.7073,0.2412,0.8657,0.3768,0.3526,0.4353,3.440


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0

In [19]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'f1')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9561,0.8891,0.5179,0.9528,0.6711,0.6498,0.6846
1,0.9534,0.8719,0.4691,0.9785,0.6341,0.6125,0.6601
2,0.9521,0.8972,0.4974,0.9065,0.6424,0.6190,0.6512
3,0.9601,0.9269,0.5590,0.9646,0.7078,0.6880,0.7176
4,0.9583,0.8887,0.5436,0.9550,0.6928,0.6722,0.7031
Mean,0.9560,0.8947,0.5174,0.9515,0.6696,0.6483,0.6833
Std,0.0030,0.0180,0.0321,0.0242,0.0283,0.0293,0.0250


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=42, reg_alpha=0.0,
                                             reg_lambda=0.0, silent='

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9521,0.8861,0.441,0.9352,0.5994,0.5774,0.6239


In [20]:
pred_holdout

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,...,YEAR_cat_3,YEAR_cat_4,YEAR_cat_5,YEAR_cat_6,YEAR_cat_7,YEAR_cat_8,YEAR_cat_9,Y_LABEL,Label,Score
0,-0.150539,1.184743,-0.337685,-0.151268,-0.125586,0.744498,-0.227563,-0.044826,1.158665,-0.084564,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.8972
1,-0.039436,-0.053225,0.122308,-0.151268,-0.138784,0.434817,-0.227563,-0.044826,1.253153,5.856518,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.8972
2,-0.354228,0.276900,-0.337685,-0.151268,-0.072793,0.521915,-0.227563,-0.044826,1.238980,-0.084564,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.8889
3,0.100637,-0.135756,-0.337685,-0.151268,-0.165180,-0.455518,-0.227563,-0.044826,-0.811419,-0.084564,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.8896
4,-0.202805,-0.218288,-0.337685,-0.151268,1.563776,0.667078,0.748473,-0.044826,-0.870137,-0.084564,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1,0.9209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814,0.637933,-0.300819,-0.337685,-0.151268,-0.151982,-0.300677,-0.227563,-0.044826,-0.704782,-0.084564,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0.8779
2815,-0.403806,-0.465881,-0.337685,-0.151268,-0.165180,-0.484550,-0.227563,-0.044826,-0.670362,-0.084564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.8927
2816,-0.328543,0.029306,-0.337685,-0.151268,-0.151982,-0.561971,-0.227563,-0.044826,-0.797921,-0.084564,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.8794
2817,-0.020023,-0.548412,-0.337685,-0.151268,-0.165180,-0.474873,-0.227563,-0.044826,-0.867437,-0.084564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.8823


In [ ]:
final_model = finalize_model(blended)

In [23]:
prep_pipe = get_config("prep_pipe")

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_cat',
                                                            'YEAR_cat'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'SAMPLE_TRANSFER_DAY',
                                                          'ANONYMOUS_2', 'AG',
                                                          'AL', 'B', 'BA', 'BE',
                                                          'CA', 'CO', 'CR',
                                                          'CU', 'FE', 'H2O',


In [25]:
prep_pipe.steps.append(['trained_model', final_model])

In [30]:
teacher_X['teacher_prob'] = prep_pipe.predict_proba(teacher_X)[:, 1]
teacher_X['teacher_prob']

0        0.100579
1        0.109216
2        0.922157
3        0.209304
4        0.114439
           ...   
14090    0.112411
14091    0.103762
14092    0.231242
14093    0.142120
14094    0.106925
Name: teacher_prob, Length: 14095, dtype: float64

In [ ]:
total_y = test.copy()
total_y['COMPONENT_ARBITRARY_cat'] = le1.fit_transform(total_y['COMPONENT_ARBITRARY'])
total_y['YEAR_cat'] = le2.fit_transform(total_y['YEAR'])
total_y.drop(['YEAR','COMPONENT_ARBITRARY'],axis=1,inplace=True)

In [35]:
total_y

,ID,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,COMPONENT_ARBITRARY_cat,YEAR_cat
0,TEST_0000,2192,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091,0,9
1,TEST_0001,2794,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12,2,4
2,TEST_0002,1982,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714,1,3
3,TEST_0003,1404,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94,2,2
4,TEST_0004,8225,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,1714,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163,2,7
6037,TEST_6037,4131,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13,2,9
6038,TEST_6038,4325,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13,2,7
6039,TEST_6039,1364,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478,1,6


In [36]:
total_y.drop('ID',axis=1,inplace=True)

In [37]:
no_list = []
for item in teacher_X.columns:
    if item not in total_y.columns:
        if item != 'teacher_prob':
            no_list.append(item)
no_list

['SAMPLE_TRANSFER_DAY',
 'AL',
 'B',
 'BA',
 'BE',
 'CA',
 'LI',
 'MG',
 'NA',
 'P',
 'PB',
 'S',
 'SB',
 'SI',
 'SN',
 'Y_LABEL']

In [38]:
student_X = teacher_X.drop(no_list,axis=1)
student_X.shape

(14095, 19)

In [39]:
total_y.shape

(6041, 18)

In [41]:
numeric_features = []

for item in student_X.columns:
    if item not in categorical_features:
        if item != 'teacher_prob':
            numeric_features.append(item)

In [42]:
numeric_features

['ANONYMOUS_1',
 'ANONYMOUS_2',
 'AG',
 'CO',
 'CR',
 'CU',
 'FE',
 'H2O',
 'MN',
 'MO',
 'NI',
 'PQINDEX',
 'TI',
 'V',
 'V40',
 'ZN']

In [ ]:
from pycaret.regression import *

clf = setup(student_X, 
            target = 'teacher_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7,
            session_id = 42,
            use_gpu=True)

In [46]:
best5 = compare_models(fold = 5, sort = 'MAE', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0572,0.0303,0.1739,-0.0579,0.1204,0.1507,0.218
gbr,Gradient Boosting Regressor,0.0820,0.0268,0.1636,0.0639,0.1135,0.4169,1.674
lightgbm,Light Gradient Boosting Machine,0.0826,0.0273,0.1652,0.0454,0.1154,0.4199,0.194
ridge,Ridge Regression,0.0845,0.0278,0.1665,0.0310,0.1155,0.4310,0.026
lar,Least Angle Regression,0.0845,0.0278,0.1665,0.0299,0.1156,0.4310,0.026
lr,Linear Regression,0.0845,0.0278,0.1665,0.0310,0.1155,0.4310,0.032
br,Bayesian Ridge,0.0847,0.0278,0.1664,0.0310,0.1154,0.4341,0.030
omp,Orthogonal Matching Pursuit,0.0857,0.0282,0.1676,0.0171,0.1164,0.4441,0.014
knn,K Neighbors Regressor,0.0860,0.0315,0.1774,-0.1016,0.1263,0.4394,0.852
en,Elastic Net,0.0877,0.0287,0.1691,-0.0006,0.1177,0.4621,0.114


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:[HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False), GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              import

In [50]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)
final_model = finalize_model(blended)
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0763,0.0267,0.1635,0.0543,0.1128,0.3603
1,0.0723,0.0238,0.1544,0.0643,0.1070,0.3510
2,0.0731,0.0253,0.1592,0.0507,0.1099,0.3525
3,0.0786,0.0288,0.1696,0.0622,0.1166,0.3565
4,0.0796,0.0306,0.1749,0.0467,0.1197,0.3523
Mean,0.0760,0.0271,0.1643,0.0556,0.1132,0.3545
Std,0.0029,0.0024,0.0073,0.0067,0.0045,0.0034


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('huber',
                             HuberRegressor(alpha=0.0001, epsilon=1.35,
                                            fit_intercept=True, max_iter=100,
                                            tol=1e-05, warm_start=False)),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0721,0.0244,0.1561,0.0519,0.1078,0.3529


INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('huber',
                             HuberRegressor(alpha=0.0001, epsilon=1.35,
                                            fit_intercept=True, max_iter=100,
                                            tol=1e-05, warm_start=False)),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
             

In [73]:
prep_pipe.predict(student_X)

array([0.14570406, 0.13286022, 0.17756436, ..., 0.16077058, 0.18780781,
       0.14500952])

In [57]:
train_y = train['Y_LABEL']
train_y

0        0
1        0
2        1
3        0
4        0
        ..
14090    0
14091    0
14092    0
14093    0
14094    0
Name: Y_LABEL, Length: 14095, dtype: int64

In [78]:
scores = []
TP = []
FP = []
FN = []
TN = []
best_threshold = 0
best_f1_score = 0

for threshold in range(10,100,5):
    threshold = threshold/100
    pred = student_X['teacher_prob']
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(train_y, pred)
    scores.append(score)
    TP.append(confusion_matrix(train_y, pred)[0][0])
    FN.append(confusion_matrix(train_y, pred)[0][1])
    FP.append(confusion_matrix(train_y, pred)[1][0])
    TN.append(confusion_matrix(train_y, pred)[1][1])
    if score > best_f1_score:
        best_f1_score = score
        best_threshold = threshold





In [79]:
print(best_f1_score)
print(best_threshold)

0.9626955475330927
0.25


In [58]:
prep_pipe.predict(student_X)

array([0.14570406, 0.13286022, 0.17756436, ..., 0.16077058, 0.18780781,
       0.14500952])

In [80]:
pred = prep_pipe.predict(total_y)

In [76]:
sub = pd.read_csv(dpath+'sample_submission.csv')

In [81]:
sub['Y_LABEL'] = np.where(pred >=0.25,1,0)
sub

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [83]:
sub['Y_LABEL'].value_counts()

0    5976
1      65
Name: Y_LABEL, dtype: int64

In [85]:
train['Y_LABEL'].value_counts()

0    12892
1     1203
Name: Y_LABEL, dtype: int64

In [86]:
sub.to_csv(dpath+'submission/KD_with_full_submission.csv',index=False)